# Predicción del rendimiento de estudiantes usando redes neuronales

## Juan Sebastian Sierra - 2343656
## Jhoan Sebastian Fernandez - 2222772
## Luis Felipe García - 2343105
## David Alejandro Enciso Gutierrez - 2240581

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, recall_score, mean_absolute_error
warnings.filterwarnings('ignore')

## 1. Lectura de los datos

Se cargan los datos en un DataFrame y se emplean distintas funciones para caracterizarlos y obtener información relevante, que permitirá posteriormente normalizarlos y codificarlos para el entrenamiento de la red.

In [ ]:
dataFrame = pd.read_csv('student_performance.csv')
dataFrame

,sex,age,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,internet,romantic,goout,Walc,health,approved
0,F,18,GT3,A,4,4,at_home,teacher,2,2,0,no,no,4,1,3,0
1,F,17,GT3,T,1,1,at_home,other,1,2,0,yes,no,3,1,3,0
2,F,15,LE3,T,1,1,at_home,other,1,2,3,yes,no,2,3,3,1
3,F,15,GT3,T,4,2,health,services,1,3,0,yes,yes,2,1,5,1
4,F,16,GT3,T,3,3,other,other,1,2,0,no,no,2,2,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,F,19,GT3,T,2,3,services,other,1,3,1,yes,no,2,2,5,1
1040,F,18,LE3,T,3,1,teacher,services,1,2,0,yes,no,4,1,1,1
1041,F,18,GT3,T,1,1,other,other,2,2,0,no,no,1,1,5,0
1042,M,17,LE3,T,3,1,services,services,2,1,0,yes,no,5,4,2,1


In [ ]:
dataFrame.describe()

,age,Medu,Fedu,traveltime,studytime,failures,goout,Walc,health,approved
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000
mean,16.726054,2.603448,2.387931,1.522989,1.970307,0.264368,3.156130,2.284483,3.543103,0.779693
std,1.239975,1.124907,1.099938,0.731727,0.834353,0.656142,1.152575,1.285105,1.424703,0.414652
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,16.000000,2.000000,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,3.000000,2.000000,4.000000,1.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,4.000000,3.000000,5.000000,1.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,1.000000


In [ ]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sex         1044 non-null   object
 1   age         1044 non-null   int64 
 2   famsize     1044 non-null   object
 3   Pstatus     1044 non-null   object
 4   Medu        1044 non-null   int64 
 5   Fedu        1044 non-null   int64 
 6   Mjob        1044 non-null   object
 7   Fjob        1044 non-null   object
 8   traveltime  1044 non-null   int64 
 9   studytime   1044 non-null   int64 
 10  failures    1044 non-null   int64 
 11  internet    1044 non-null   object
 12  romantic    1044 non-null   object
 13  goout       1044 non-null   int64 
 14  Walc        1044 non-null   int64 
 15  health      1044 non-null   int64 
 16  approved    1044 non-null   int64 
dtypes: int64(10), object(7)
memory usage: 138.8+ KB


## 2. Separación del conjunto de entrenamiento y de prueba

A continuación se hace la separación del conjunto de entrenamiento y conjunto de prueba de manera aleatoria.

**Nota:** Se utilizó random_state=42, para que los conjuntos aleatorios sean siempre los mismos cada vez que se ejecuta el notebook y, de esta forma, poder llevar a cabo el análisis sobre una configuración estática de los datos e hiperparámetros. De esta forma, el profesor podrá obtener por su cuenta los resultados y que estos no difieran de los nuestros.

In [ ]:
train_data, test_data = sklearn.model_selection.train_test_split(dataFrame, test_size=0.2, random_state=42)
print(len(train_data), len(test_data))

835 209


In [ ]:
train_data.head(5)

,sex,age,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,internet,romantic,goout,Walc,health,approved
531,M,16,GT3,T,2,1,at_home,other,4,1,0,no,no,1,1,2,1
256,F,17,LE3,T,4,2,teacher,services,1,4,0,yes,no,3,1,4,1
350,M,19,GT3,T,1,1,other,services,3,2,3,yes,no,4,3,2,0
60,F,16,GT3,T,4,4,health,teacher,1,2,0,no,no,4,3,4,1
930,M,15,GT3,T,1,2,other,services,3,2,0,no,no,5,3,5,1


## 3. Normalización y codificación de los datos

### 3.1 Pipeline para atributos categóricos

Se crea un pipeline para codificar los atributos categóricos usando la técnica OneHotEncoding vista en clase.

In [ ]:
cat_attribs = ['sex', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'internet', 'romantic']

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("cat_encoder", OneHotEncoder(sparse_output=False))
])

### 3.2 Pipeline para atributos numéricos

Se crea un pipeline para normalizar los atributos numéricos usando StandardScaler.

In [ ]:
num_attribs = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'goout', 'Walc', 'health']

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

### 3.3 Pipeline completo

In [ ]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

### 3.4 Separación del X_train y Y_train

In [ ]:
X_train = full_pipeline.fit_transform(train_data)
X_train[0,:]

array([-0.58511174, -0.56825442, -1.28210135,  3.31007524, -1.14639316,
       -0.37848294, -1.88202685, -1.02310781, -1.07389358,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  1.        ,  0.        ])

In [ ]:
X_train.shape

(835, 29)

In [ ]:
Y_train = train_data["approved"]
Y_train

,approved
531,1
256,1
350,0
60,1
930,1
...,...
87,1
330,0
466,1
121,1


## 4. Entrenamiento de la red

In [ ]:
model = MLPClassifier(max_iter=1000, random_state=42)

param_grid = {
    'activation': ['relu', 'tanh', 'logistic'],
    'hidden_layer_sizes': [(10), (25, 30), (5, 10), (20, 20, 20)],
    'solver': ['adam', 'lbfgs', 'sgd'],
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=10,
    cv=5,
    random_state=1
)

random_search.fit(X_train, Y_train)
results = pd.DataFrame(random_search.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_hidden_layer_sizes,param_activation,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.746376,0.104720,0.001392,0.000082,adam,"(5, 10)",logistic,"{'solver': 'adam', 'hidden_layer_sizes': (5, 1...",0.832335,0.832335,0.766467,0.808383,0.832335,0.814371,0.025686,3
1,1.856945,0.474183,0.003811,0.002299,lbfgs,"(20, 20, 20)",logistic,"{'solver': 'lbfgs', 'hidden_layer_sizes': (20,...",0.682635,0.736527,0.688623,0.754491,0.736527,0.719760,0.028693,10
2,0.837812,0.386079,0.001913,0.000088,lbfgs,"(25, 30)",logistic,"{'solver': 'lbfgs', 'hidden_layer_sizes': (25,...",0.742515,0.730539,0.682635,0.748503,0.718563,0.724551,0.023346,8
3,1.419256,0.077995,0.001420,0.000193,adam,"(25, 30)",relu,"{'solver': 'adam', 'hidden_layer_sizes': (25, ...",0.772455,0.760479,0.682635,0.730539,0.766467,0.742515,0.033232,7
4,2.154025,0.678551,0.001900,0.000211,lbfgs,"(5, 10)",tanh,"{'solver': 'lbfgs', 'hidden_layer_sizes': (5, ...",0.754491,0.754491,0.748503,0.784431,0.724551,0.753293,0.019087,6
5,2.531142,0.704946,0.003552,0.001158,sgd,"(25, 30)",tanh,"{'solver': 'sgd', 'hidden_layer_sizes': (25, 3...",0.826347,0.844311,0.772455,0.820359,0.820359,0.816766,0.023832,2
6,3.877540,1.327038,0.002549,0.000483,adam,"(20, 20, 20)",tanh,"{'solver': 'adam', 'hidden_layer_sizes': (20, ...",0.742515,0.736527,0.694611,0.718563,0.718563,0.722156,0.016766,9
7,1.385927,0.285751,0.002031,0.000019,sgd,"(20, 20, 20)",tanh,"{'solver': 'sgd', 'hidden_layer_sizes': (20, 2...",0.814371,0.844311,0.790419,0.826347,0.820359,0.819162,0.017519,1
8,0.129874,0.005278,0.001474,0.000070,sgd,"(25, 30)",logistic,"{'solver': 'sgd', 'hidden_layer_sizes': (25, 3...",0.796407,0.796407,0.796407,0.796407,0.796407,0.796407,0.000000,4
9,0.184640,0.029298,0.001395,0.000108,sgd,10,logistic,"{'solver': 'sgd', 'hidden_layer_sizes': 10, 'a...",0.796407,0.796407,0.796407,0.796407,0.796407,0.796407,0.000000,4


In [ ]:
cols = ['param_hidden_layer_sizes', 'param_activation', 'param_solver', 'mean_test_score', 'rank_test_score']
results[cols].sort_values(by='rank_test_score')

,param_hidden_layer_sizes,param_activation,param_solver,mean_test_score,rank_test_score
7,"(20, 20, 20)",tanh,sgd,0.819162,1
5,"(25, 30)",tanh,sgd,0.816766,2
0,"(5, 10)",logistic,adam,0.814371,3
8,"(25, 30)",logistic,sgd,0.796407,4
9,10,logistic,sgd,0.796407,4
4,"(5, 10)",tanh,lbfgs,0.753293,6
3,"(25, 30)",relu,adam,0.742515,7
2,"(25, 30)",logistic,lbfgs,0.724551,8
6,"(20, 20, 20)",tanh,adam,0.722156,9
1,"(20, 20, 20)",logistic,lbfgs,0.719760,10


Usaremos las 5 mejores combinaciones de hiperparámetros para calcular el accuracy en la siguiente sección.

## 5. Cálculo del Accuracy

In [ ]:
X_test = full_pipeline.transform(test_data)
Y_test = test_data["approved"]

In [ ]:
model1 = MLPClassifier(activation='tanh', hidden_layer_sizes=(20, 20, 20), solver='sgd')
model2 = MLPClassifier(activation='tanh', hidden_layer_sizes=(25, 30), solver='sgd')
model3 = MLPClassifier(activation='logistic', hidden_layer_sizes=(5, 10), solver='adam')
model4 = MLPClassifier(activation='logistic', hidden_layer_sizes=(25, 30), solver='sgd')
model5 = MLPClassifier(activation='logistic', hidden_layer_sizes=10, solver='sgd')

In [ ]:
predict1 = model1.fit(X_train, Y_train).predict(X_test)
predict2 = model2.fit(X_train, Y_train).predict(X_test)
predict3 = model3.fit(X_train, Y_train).predict(X_test)
predict4 = model4.fit(X_train, Y_train).predict(X_test)
predict5 = model5.fit(X_train, Y_train).predict(X_test)

In [ ]:
accuracy1 = accuracy_score(Y_test, predict1)
sensibility1 = recall_score(Y_test, predict1, pos_label=1)
specificity1 = recall_score(Y_test, predict1, pos_label=0)
mae1 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict1
       )

accuracy2 = accuracy_score(Y_test, predict2)
sensibility2 = recall_score(Y_test, predict2, pos_label=1)
specificity2 = recall_score(Y_test, predict2, pos_label=0)
mae2 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict2
       )

accuracy3 = accuracy_score(Y_test, predict3)
sensibility3 = recall_score(Y_test, predict3, pos_label=1)
specificity3 = recall_score(Y_test, predict3, pos_label=0)
mae3 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict3
       )

accuracy4 = accuracy_score(Y_test, predict4)
sensibility4 = recall_score(Y_test, predict4, pos_label=1)
specificity4 = recall_score(Y_test, predict4, pos_label=0)
mae4 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict4
       )

accuracy5 = accuracy_score(Y_test, predict5)
sensibility5 = recall_score(Y_test, predict5, pos_label=1)
specificity5 = recall_score(Y_test, predict5, pos_label=0)
mae5 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict5
       )

In [ ]:
final_results = {
    'Modelo': ['1','2','3','4','5'],
    'Accuracy': [accuracy1, accuracy2, accuracy3, accuracy4, accuracy5],
    'Sensibilidad': [sensibility1, sensibility2, sensibility3, sensibility4, sensibility5],
    'Especificidad': [specificity1, specificity2, specificity3, specificity4, specificity5],
    'MAE': [mae1, mae2, mae3, mae4, mae5]
}

result_frame = pd.DataFrame(final_results)

result_frame = result_frame.round(5)
result_frame

,Modelo,Accuracy,Sensibilidad,Especificidad,MAE
0,1,0.73684,0.91946,0.28333,0.26316
1,2,0.71770,0.92617,0.20000,0.28230
2,3,0.71292,1.00000,0.00000,0.28708
3,4,0.71292,1.00000,0.00000,0.28708
4,5,0.71292,1.00000,0.00000,0.28708


Los hiperparámetros que, de momento, permiten tener un mayor Accuracy son:

- activation = 'tanh'
- hidden_layer_sizes = (25, 30)
- solver = 'sgd'

## 6. Uso de otro hiperparámetro

Se añade el hiperarámetro alpha al modelo que mejor Accuracy ha demostrado, con el fin de evitar overfitting o memorización en lugar de aprendizaje.

In [ ]:
model6 = MLPClassifier(activation='tanh', hidden_layer_sizes=(25, 30), solver='sgd', alpha=0.1)
model7 = MLPClassifier(activation='tanh', hidden_layer_sizes=(25, 30), solver='sgd', alpha=1)

In [ ]:
predict6 = model6.fit(X_train, Y_train).predict(X_test)
predict7 = model7.fit(X_train, Y_train).predict(X_test)

In [ ]:
accuracy6 = accuracy_score(Y_test, predict6)
sensibility6 = recall_score(Y_test, predict6, pos_label=1)
specificity6 = recall_score(Y_test, predict6, pos_label=0)
mae6 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict6
       )

accuracy7 = accuracy_score(Y_test, predict7)
sensibility7 = recall_score(Y_test, predict7, pos_label=1)
specificity7 = recall_score(Y_test, predict7, pos_label=0)
mae7 = mean_absolute_error(
        y_true  = Y_test,
        y_pred  = predict7
       )

In [ ]:
final_results = {
    'Modelo': ['1','2','3','4','5','6','7'],
    'Accuracy': [accuracy1, accuracy2, accuracy3, accuracy4, accuracy5, accuracy6, accuracy7],
    'Sensibilidad': [sensibility1, sensibility2, sensibility3, sensibility4, sensibility5, sensibility6, sensibility7],
    'Especificidad': [specificity1, specificity2, specificity3, specificity4, specificity5, specificity6, specificity7],
    'MAE': [mae1, mae2, mae3, mae4, mae5, mae6, mae7]
}

result_frame = pd.DataFrame(final_results)

result_frame = result_frame.round(5)
result_frame

,Modelo,Accuracy,Sensibilidad,Especificidad,MAE
0,1,0.73684,0.91946,0.28333,0.26316
1,2,0.71770,0.92617,0.20000,0.28230
2,3,0.71292,1.00000,0.00000,0.28708
3,4,0.71292,1.00000,0.00000,0.28708
4,5,0.71292,1.00000,0.00000,0.28708
5,6,0.73206,0.93960,0.21667,0.26794
6,7,0.73206,0.94631,0.20000,0.26794


## 7. Análisis

En este caso particular, la adición de regularización a través del parámetro alpha no parece mejorar el rendimiento del modelo en el conjunto de prueba; de hecho, lo degrada ligeramente en términos de Accuracy y Sensibilidad. Esto podría sugerir que el modelo base (Modelo 2) no estaba sobreajustado significativamente, o que los valores de alpha probados no son los óptimos para este conjunto de datos y configuración de red. En este escenario, el Modelo 2 sigue siendo el de mejor rendimiento entre los modelos que has evaluado hasta ahora. Para una exploración más exhaustiva, se podrían probar otros rangos o valores de alpha en combinación con una búsqueda de hiperparámetros más amplia.